In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# !python -m pip freeze
!python -c "import sys; print(sys.executable)"
import sys
import os
print(sys.executable)
print(os.environ['VIRTUAL_ENV'])

D:\MyProjects\Study_at_HUST\2020-2\thi_giac_may_tinh\image_retrieval\venv\Scripts\python.exe
D:\MyProjects\Study_at_HUST\2020-2\thi_giac_may_tinh\image_retrieval\venv\Scripts\python.exe
D:\MyProjects\Study_at_HUST\2020-2\thi_giac_may_tinh\image_retrieval\venv


In [3]:
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms

cuda = torch.cuda.is_available()

In [4]:
from datasets import IRDataset

mean, std = 0.1307, 0.3081

triplet_train_dataset = IRDataset(descriptor_path = './data/cifar-10/train.json',
                                transform=transforms.Compose([
                                 transforms.ToTensor(),
                                 transforms.Normalize((mean,), (std,))
                             ]))
triplet_test_dataset = IRDataset(descriptor_path = './data/cifar-10/test.json',
                                transform=transforms.Compose([
                                 transforms.ToTensor(),
                                 transforms.Normalize((mean,), (std,))
                             ]))

batch_size = 2

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

triplet_train_loader = torch.utils.data.DataLoader(triplet_train_dataset, batch_size=batch_size, shuffle=True, **kwargs)
triplet_test_loader = torch.utils.data.DataLoader(triplet_test_dataset, batch_size=batch_size, shuffle=False, **kwargs)

ModuleNotFoundError: No module named 'datasets'

In [ ]:
print([i.shape for i in triplet_train_dataset.__getitem__(0)[0]])

In [ ]:
from networks import EmbeddingNet, EmbeddingNetL2, TripletNet
from losses import TripletLoss
from trainer import fit
import torch.nn as nn

embedding_net = EmbeddingNetL2()
triplet_net = TripletNet(embedding_net)

if cuda:
    triplet_net = triplet_net.cuda()

margin = 1.0
loss_fn = TripletLoss(margin)

# def l_infinity(x1, x2):
#     return torch.max(torch.abs(x1 - x2), dim=1).values
# loss_fn = nn.TripletMarginWithDistanceLoss(distance_function=l_infinity, margin=1.5)

lr = 1e-3
optimizer = optim.Adam(triplet_net.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
n_epochs = 20
log_interval = 100

In [ ]:
with torch.autograd.set_detect_anomaly(True):
    fit(triplet_train_loader, triplet_test_loader, triplet_net, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval)

In [ ]:
# sample = triplet_train_dataset.__getitem__(0)[0]
# dataloader_iterator = iter(triplet_train_loader)
# sample, _ = next(dataloader_iterator)
# print(sample)

In [ ]:
# e = triplet_net.forward(sample[0], sample[1], sample[2])

In [ ]:
# import torch.nn.functional as F
# pos_dis = (e[0] - e[1]).pow(2).sum(1)
# neg_dis = (e[0] - e[2]).pow(2).sum(1)
# losses = F.relu(pos_dis - neg_dis + 1)
# print(pos_dis, neg_dis, losses)

In [ ]:
# losses.backward()

In [ ]:
# triplet_net.parameters

In [ ]:
!python main.py